In [1]:
import os
import shutil
import opendatasets as od
import pandas as pd
from IPython.display import display
from database.sql_connector import SqlConnector

In [2]:
sql_con = SqlConnector()

Databases configured
Tables configured
Views configured
User configured


## Netflix Originals

In [3]:
kaggle_user = "luiscorter"
folder_name = "netflix-original-films-imdb-scores"

In [4]:
od.download(f"https://www.kaggle.com/{kaggle_user}/{folder_name}", force=True)

100%|██████████| 12.4k/12.4k [00:00<00:00, 9.41MB/s]

In [6]:
for ind, file in enumerate(os.listdir(folder_name)):
    df = pd.read_csv(os.path.join(folder_name,file), encoding="ISO-8859-1")
    df.Language = df.Language.apply(lambda x: x.split('/'))
    df.Genre = df.Genre.apply(lambda x: x.split('/'))
    df = df.explode('Language')
    df = df.explode('Genre')
    df.rename(columns={'IMDB Score':'IMDB_Score'}, inplace=True)
    df.Premiere = df.Premiere.str.replace(".", ",", regex=True)
    df.Premiere = pd.to_datetime(df.Premiere, format="%B %d, %Y")
    sql_con.run_query("""DELETE FROM KAGGLE.NETFLIX_ORIGINALS.DATA""")
    sql_con.ingest_dataframe(df, schema='NETFLIX_ORIGINALS', table='DATA')

shutil.rmtree(folder_name)

Query done
Data ingested
